In [8]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, FFMpegWriter
from matplotlib.ticker import FuncFormatter
from IPython.display import HTML

# =========================================================
# CONFIGURATION
# =========================================================
FPS = 30
ANIMATION_DURATION = 10     # seconds (movement)
PAUSE_DURATION = 5          # seconds (final pause)

# =========================================================
# TIME AXIS
# =========================================================
years = np.array([2023, 2024, 2025], dtype=object)

# Equal spacing for categorical years
x_pos = np.arange(len(years))
x_shifted = x_pos - 0.5     # shift values between gridlines

# =========================================================
# DATA SERIES (PERCENT VALUES)
# =========================================================
series_a = np.array([49, 56, 63])
series_b = np.array([62, 35, 72])
series_c = np.array([90, 50, 100])
series_d = np.array([85, 100, 83])

# Special series: appears only at final year
series_e = np.array([np.nan, np.nan, 26])

# =========================================================
# FRAME CALCULATIONS
# =========================================================
total_frames = ANIMATION_DURATION * FPS
frames_per_step = total_frames // (len(years) - 1)
pause_frames = PAUSE_DURATION * FPS
total_frames_with_pause = total_frames + pause_frames

# =========================================================
# FIGURE & AXES SETUP
# =========================================================
fig, ax = plt.subplots(figsize=(13, 5))
ax.set_position([0.05, 0.1, 0.93, 0.83])

ax.set_xlim(-1, len(years) - 1)
ax.set_ylim(-20, 109)
ax.set_yticks(np.arange(-20, 101, 20))

# Grid & zero line
ax.grid(True, alpha=0.3, color="#404040")
ax.axhline(0, color="black", linewidth=1, alpha=0.7)

# Background
fig.patch.set_facecolor("#eaeaef")
ax.set_facecolor("#eaeaef")

# X axis formatting
ax.set_xticks(x_pos)
ax.set_xticks(x_shifted, minor=True)
ax.set_xticklabels(years, minor=True)

ax.tick_params(axis="x", which="major", bottom=False, labelbottom=False)
ax.tick_params(axis="x", which="minor", pad=10, colors="#404040")
ax.tick_params(axis="y", pad=10)

# Gridlines on years
ax.grid(True, axis="x", which="major", color="#555557", linewidth=1)

# Spines
ax.spines["top"].set_visible(False)
for spine in ax.spines.values():
    spine.set_color((0.25, 0.25, 0.25, 0.3))
    spine.set_linewidth(1)

# Tick label color
for tick in ax.xaxis.get_ticklabels() + ax.yaxis.get_ticklabels():
    tick.set_color("#555557")

# =========================================================
# LINE DEFINITIONS
# =========================================================
def create_line(color):
    return ax.plot([], [], marker="o", linewidth=3,
                   markersize=17, markerfacecolor="white",
                   markeredgewidth=6, color=color)[0]

line_a = create_line("#C82E3F")
line_b = create_line("#96232F")
line_c = create_line("#FF0000")
line_d = create_line("#FFC000")
line_e = create_line("#B7620C")

pt_a = create_line("#C82E3F")
pt_b = create_line("#96232F")
pt_c = create_line("#FF0000")
pt_d = create_line("#FFC000")
pt_e = create_line("#B7620C")

# =========================================================
# INITIALIZATION
# =========================================================
def init():
    for artist in (line_a, line_b, line_c, line_d, line_e,
                   pt_a, pt_b, pt_c, pt_d, pt_e):
        artist.set_data([], [])
    return line_a, line_b, line_c, line_d, line_e, pt_a, pt_b, pt_c, pt_d, pt_e

# =========================================================
# ANIMATION FUNCTION
# =========================================================
def animate(frame):

    last_motion_frame = (len(years) - 1) * frames_per_step - 1
    if frame >= last_motion_frame:
        frame = last_motion_frame

    idx = frame / frames_per_step
    lower = int(np.floor(idx))
    upper = min(lower + 1, len(years) - 1)
    frac = idx - lower

    x_now = x_shifted[lower] + frac * (x_shifted[upper] - x_shifted[lower])

    def interp(data):
        return data[lower] + frac * (data[upper] - data[lower])

    y_a = interp(series_a)
    y_b = interp(series_b)
    y_c = interp(series_c)
    y_d = interp(series_d)

    x_draw = np.append(x_shifted[:upper], x_now)

    line_a.set_data(x_draw, np.append(series_a[:upper], y_a))
    line_b.set_data(x_draw, np.append(series_b[:upper], y_b))
    line_c.set_data(x_draw, np.append(series_c[:upper], y_c))
    line_d.set_data(x_draw, np.append(series_d[:upper], y_d))

    pt_a.set_data([x_now], [y_a])
    pt_b.set_data([x_now], [y_b])
    pt_c.set_data([x_now], [y_c])
    pt_d.set_data([x_now], [y_d])

    # Special series: appears only at final year
    if frame >= last_motion_frame:
        line_e.set_data([x_shifted[-1]], [series_e[-1]])
        pt_e.set_data([x_shifted[-1]], [series_e[-1]])
    else:
        line_e.set_data([], [])
        pt_e.set_data([], [])

    return (line_a, line_b, line_c, line_d, line_e,
            pt_a, pt_b, pt_c, pt_d, pt_e)

# =========================================================
# BUILD & EXPORT ANIMATION
# =========================================================
animation = FuncAnimation(
    fig,
    animate,
    init_func=init,
    frames=total_frames_with_pause,
    interval=1000 / FPS,
    blit=False,
    repeat=True
)

# gif_path = "animation.gif"

# gif_writer = PillowWriter(
#     fps=FPS,
#     metadata={"artist": "Matplotlib"},
#     bitrate=1800
# )

# animation.save(gif_path, writer=gif_writer)
# print(f"✅ GIF saved: {gif_path}")

# # Save MP4
# writer = FFMpegWriter(
#     fps=FPS,
#     codec="libx264",
#     bitrate=2500,
#     extra_args=["-pix_fmt", "yuv420p"]
# )

# animation.save("animation.mp4", writer=writer, dpi=300)
# print("✅ MP4 saved: animation.mp4")

plt.close(fig)
HTML(animation.to_html5_video())


✅ GIF saved: animation.gif
✅ MP4 saved: animation.mp4


In [9]:
# from google.colab import files
# files.download('animation.gif')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>